In [1]:
cd /Users/keonkookkim/Documents

/Users/keonkookkim/Documents


In [2]:
!xattr -d com.apple.quarantine chromedriver # 맥에서 크롬 브라우저 허용하는 코드 

xattr: chromedriver: No such xattr: com.apple.quarantine


In [3]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys # enter 치는 명령어
import csv 
import pandas as pd 

In [4]:
# 브라우저 생성 
browser = webdriver.Chrome('/Users/keonkookkim/Documents/chromedriver')

# 웹 사이트 열기 
browser.get('https://www.yogiyo.co.kr/mobile/#/')

In [5]:
# 검색창 클릭
search = browser.find_element_by_xpath('//*[@id="search"]/div/form/input')
search.click()

In [6]:
# 검색어 입력
search.send_keys('서울특별시 서초구 반포동 20-45 반포자이플라자')
search.send_keys(Keys.ENTER)
time.sleep(15)

In [15]:
# 스크롤 전 높이
before_h = browser.execute_script("return window.scrollY") # 현재 스크롤 된 높이를 계산

# 무한 반복문 사용
while True:
    # 맨 아래로 스크롤을 내린다.
    browser.find_element_by_css_selector("body").send_keys(Keys.END) # end키를 통해 스크롤을 맨 아래로 내림 
    
    # 스크롤 사이 페이지 로딩 시간 추가
    time.sleep(3)
    
    # 스크롤 후 높이 확인 
    after_h = browser.execute_script("return window.scrollY")
    
    # before_h = after_h # after의 높이를 before로 갱신
    
    if after_h > 10000: # 스크롤 높이보다 높으면 멈춤 
        break
    
    ########################### 스크롤 높이 설정 필요 ###########################

In [97]:
# # 가게 클릭 
# search = browser.find_element_by_xpath('//*[@id="content"]/div/div[3]/div/div[2]/div[1]/div')
# search.click()
# time.sleep(10)


In [98]:
# # 리뷰탭 클릭 
# search = browser.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a')
# search.click()
# time.sleep(10)

In [39]:
# star_count = 0
# for j in range(1,6):
#     star = browser.find_element_by_xpath(f'//*[@id="review"]/li[16]/div[2]/div/span[1]/span[{j}]').get_attribute("class")
#     if star == 'full ng-scope':
#         star_count += 1
        
#     else :
#         star_count += 0
# print(star_count)

5


# 실행시 파일 초기화됨 주의!!!

In [10]:
# 파일 생성 
f = open(r"/Users/keonkookkim/Desktop/KKK/Selenium/valid_data.csv", 'w', encoding="UTF-8", newline="") # 파일 입출력 기능 

csvWirter = csv.writer(f) # wirter에 파일(f) 넘겨주기
f.close()

In [168]:
# # 크롤링 하면서 더보기 누르는 버전 

# def review_crawling():
#     count = browser.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div/strong[1]').text
    
#     time.sleep(2)
#     num = 0
#     for i in range(2, int(count)+1):
#         # 가게 이름
#         title = browser.find_element_by_css_selector('span.restaurant-name.ng-binding').text
        
#         items = browser.find_element_by_xpath(f'//*[@id="review"]/li[{i}]')
#         review = items.find_element_by_css_selector("p.ng-binding").text
#         ########################## 평점 추가 고민 해보기 ##########################
        
#         print(title,review)
#         csvWirter.writerow([title, review]) # 행 추가
#         num += 1
#         time.sleep(0.5)
#         if num % 10 == 0 :
#             browser.find_element_by_css_selector("body").send_keys(Keys.END) # end키를 통해 스크롤을 맨 아래로 내림 
#             time.sleep(5)
#             # 더보기 클릭
#             more_click = browser.find_element_by_css_selector('li.list-group-item.btn-more')
#             more_click.click()
#             time.sleep(5)  

----
### 리뷰 크롤링
----

In [11]:
# 더보기를 미리 다 눌러놓고 크롤링하는 버전 
# 이게 휠씬 빠름

def reveiw_crawling_first_scroll():
    while True:
        try:          
            browser.find_element_by_css_selector("body").send_keys(Keys.END) # end키를 통해 스크롤을 맨 아래로 내림 
            time.sleep(1)
            
            # 더보기 클릭
            more_click = browser.find_element_by_css_selector('li.list-group-item.btn-more')
            isExistNextPage = more_click.is_enabled() # 더보기 클릭 가능하다면
            if (isExistNextPage == True):
                more_click.click()
                time.sleep(2)  
            else :  # 없다면 끝 
                break
        except:
            break

    count = browser.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div/strong[1]').text
    if int(count) >= 2: # 리뷰 개수가 2개 이하면 크롤링X
        for i in range(2, int(count)+1):
            # 가게 이름
            title = browser.find_element_by_css_selector('span.restaurant-name.ng-binding').text
            
            items = browser.find_element_by_xpath(f'//*[@id="review"]/li[{i}]')
            review = items.find_element_by_css_selector("p.ng-binding").text
            
            # 별 개수 세기
            star_count = 0
            for j in range(1,6):
                star = browser.find_element_by_xpath(f'//*[@id="review"]/li[{i}]/div[2]/div/span[1]/span[{j}]').get_attribute("class")
                if star == 'full ng-scope':
                    star_count += 1
                    
                else :
                    star_count += 0
            
            print(title,'|',review,'|',star_count)
            csvWirter.writerow([title, review, star_count]) # 행 추가

---
### 별점 낮은것 크롤링
---

In [65]:
# def bad_reveiw_crawling():
#     while True:
#         try:          
#             browser.find_element_by_css_selector("body").send_keys(Keys.END) # end키를 통해 스크롤을 맨 아래로 내림 
#             time.sleep(1)
            
#             # 더보기 클릭
#             more_click = browser.find_element_by_css_selector('li.list-group-item.btn-more')
#             isExistNextPage = more_click.is_enabled() # 더보기 클릭 가능하다면
#             if (isExistNextPage == True):
#                 more_click.click()
#                 time.sleep(2)  
#             else :  # 없다면 끝 
#                 break
#         except:
#             break

#     count = browser.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div/strong[1]').text
#     if int(count) >= 2: # 리뷰 개수가 2개 이하면 크롤링X
#         for i in range(2, int(count)+1):
#             # 가게 이름
#             title = browser.find_element_by_css_selector('span.restaurant-name.ng-binding').text
            
#             items = browser.find_element_by_xpath(f'//*[@id="review"]/li[{i}]')
#             review = items.find_element_by_css_selector("p.ng-binding").text
            
#             # 별 개수 세기
#             star_count = 0
#             for j in range(1,6):
#                 star = browser.find_element_by_xpath(f'//*[@id="review"]/li[{i}]/div[2]/div/span[1]/span[{j}]').get_attribute("class")
#                 if star == 'full ng-scope':
#                     star_count += 1
                    
#                 else :
#                     star_count += 0
            
#             if star_count < 3 :
                
#                  # 맛 별점
#                 taste = browser.find_element_by_xpath(f'//*[@id="review"]/li[{i}]/div[2]/div/span[2]/span[3]').text
#                 # 양 별점 
#                 quantity = browser.find_element_by_xpath(f'//*[@id="review"]/li[{i}]/div[2]/div/span[2]/span[6]').text
#                 # 배달 별점
#                 delivery = quantity = browser.find_element_by_xpath(f'//*[@id="review"]/li[{i}]/div[2]/div/span[2]/span[9]').text
                
#                 print(title,'|',review,'|',star_count,'|',taste,'|',quantity,'|',delivery)
#                 csvWirter.writerow([title, review, star_count, taste, quantity, delivery]) # 행 추가

In [8]:
# 파일에 데이터 덮어쓰기
f = open(r"/Users/keonkookkim/Desktop/KKK/Selenium/valid_data.csv", 'a', encoding="UTF-8", newline="") # 파일 입출력 기능 

csvWirter = csv.writer(f) # wirter에 파일(f) 넘겨주기

---
# 지역 선택 후 전체 식당 크롤링
---

In [68]:

# 슈퍼레드위크 추천 크롤링
for i in range(1, 20):
    data = pd.read_csv('/Users/keonkookkim/Desktop/KKK/Selenium/review.csv')
    data.columns =['store','review','total_scope']
    store_list = data['store'].unique()
    try:
        # 가게 클릭 
        search = browser.find_element_by_xpath(f'//*[@id="content"]/div/div[3]/div/div[2]/div[{i}]/div')
        search.click()
        time.sleep(5)
        store_name = browser.find_element_by_css_selector('span.restaurant-name.ng-binding').text
        if store_name not in store_list:
        # 리뷰탭 클릭 
            search = browser.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a')
            search.click()
            time.sleep(5)
            
            #### 리뷰 크롤링
            reveiw_crawling_first_scroll()
            # bad_reveiw_crawling()
            
            # 뒤로가기
            browser.back()
            time.sleep(6)
        else :
            browser.back()
            time.sleep(6) 
    except:
        break

# 일반 등록 음식점 크롤링
for i in range(1,20):    
    data = pd.read_csv('/Users/keonkookkim/Desktop/KKK/Selenium/review.csv')
    data.columns =['store','review','total_scope']
    store_list = data['store'].unique()
    try:
        # 가게 클릭
        search = browser.find_element_by_xpath(f'//*[@id="content"]/div/div[4]/div/div[2]/div[{i}]/div')
        search.click()
        time.sleep(5)        
        store_name = browser.find_element_by_css_selector('span.restaurant-name.ng-binding').text
        if store_name not in store_list:
        # 리뷰탭 클릭 
            search = browser.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a')
            search.click()
            time.sleep(5)
            
            #### 리뷰 크롤링
            reveiw_crawling_first_scroll()
            # bad_reveiw_crawling()
                
            # 뒤로가기
            browser.back()
            time.sleep(6)
        else :
            browser.back()
            time.sleep(6)
            
    except:
        break

호식이두마리치킨-청룡점 | 배달 왤케 느림? 한시간반이 넘게 걸리노 | 1
호식이두마리치킨-청룡점 | 예정시간 20분 지나도 배달안와서 전화걸었으나 받지 않음. | 1
호식이두마리치킨-청룡점 | 맛있어요! 근데 배달은 2시간 10분 걸렸어요 ㅎㅎ 1시간 40분쯤 지났을 때 취소하려고 전화드렸는데 조리 중이라 하셔서 취소 못해서 아쉬운건 어쩔 수 없네요 ㅠㅠㅠㅠ 주문이 너무 많이 밀리셔서 주문 받기 힘드시면 미리 늦어질 수 있으니 취소 유무 결정해주셨으면 좋을 것 같아요 ㅎㅎㅎㅎ | 1
호식이두마리치킨-청룡점 | 배달 예상시간보다 늦고 식어서 딱딱한 상태였음. 리뷰이벤트 오지않음. | 2
호식이두마리치킨-청룡점 | 자주 시켜 먹는데 오늘은 정말 배달이 최악이네요…. 전화도 안받고… 취소 못해서 결국 받긴했는데. 다음엔 이런일 없었으면 합니다. | 1
호식이두마리치킨-청룡점 | 배달이 정말 너무 오래 걸리네요 치킨도 다 식었구요 | 1
호식이두마리치킨-청룡점 | 배달 1시간 40분 걸려서 받음. 미안하다는 말 한마디 없음 | 1
호식이두마리치킨-청룡점 | 비와서 배달늦고 맛 이 냉동식품보다 맛없는건 이해함. 그냥 먹고 넘어가면 되니까. 근데 순살치킨에 이상한 이물질을 씹고 뭔가 싶음. 대부분이 가슴살인건 알겠는데 유독 몇몇부분이 이상한게 섞인 맛임. 반죽 문제일 수도 있으나 먹다가 너무 물림. 이물질은 뼈라면 그나마 다행이겠지만 아무리 봐도 난 잘 모르겠음. 괜히 찝찝 | 1
호식이두마리치킨-청룡점 | 한번씩 시켜먹는 집인데.. 그날따라 배달 거의 2시간가까이걸림.. 그럴거면 연락을 주시지 배달갔다고 한 뒤 30분지나서옴 치킨도 좀 식었었음 말 '한마디' 면 해결될 문제를 사업하는 입장에서보면 1점, | 1
호식이두마리치킨-청룡점 | 제시간에 배달 못할 것 같으면 주문을 받으면 안됨. 뭐 사람 하는 일이니 다소는 이해할 수 있지만 배달시간 두배는 말도 안됨. 리뷰 확인 안한 제 잘못인 듯 합니다. ㅎㅎ | 1
호식이두마리치킨-청룡점 | 배달도착예상시간보다 거의 한

-----

In [27]:
# 파일에 데이터 덮어쓰기
f = open(r"/Users/keonkookkim/Desktop/KKK/Selenium/valid_data.csv", 'a', encoding="UTF-8", newline="") # 파일 입출력 기능 

csvWirter = csv.writer(f) # wirter에 파일(f) 넘겨주기

In [28]:
reveiw_crawling_first_scroll()

모던니혼일본라멘돈카츠전문점 | 점심으로 잘 먹었습니다 :) | 4
모던니혼일본라멘돈카츠전문점 | . | 5
모던니혼일본라멘돈카츠전문점 | 잘 먹었어요~! 다들 좋은 하루보냐세요 | 5
모던니혼일본라멘돈카츠전문점 | 맛있어서 자주 시켜 먹습니다👍❤ | 5
모던니혼일본라멘돈카츠전문점 | 맛있어요👍👍👍👍👍👍👍👍 | 5
모던니혼일본라멘돈카츠전문점 | 맛있게잘먹었습니다~ | 5
모던니혼일본라멘돈카츠전문점 | 처음 주문해봤는데 만족스럽네요 | 5
모던니혼일본라멘돈카츠전문점 | 차슈의 엄청난 두께에 깜짝 놀랐네요~맛있게 잘 먹었습니다~^^ | 5
모던니혼일본라멘돈카츠전문점 | 깔끔하고 맛남. 돈까스 소스가 차가운건 쫌 싫어요. 라멘 맛있네요. | 5
모던니혼일본라멘돈카츠전문점 | . | 5
모던니혼일본라멘돈카츠전문점 | 돈까스 부드럽고 치즈 양이 많아 좋아요 ㅎㅎㅎ | 5
모던니혼일본라멘돈카츠전문점 | 그냥그랬습니다 쏘쏘해요 | 3
모던니혼일본라멘돈카츠전문점 | 돈까스소스에 겨자가 들어가서 살짝 제 입맛은 아니었고 라멘에 들어간 차슈에 살짝 냄새가........좀...... | 3
모던니혼일본라멘돈카츠전문점 | 맛있어요 다음에 또 주문할게요 | 5
모던니혼일본라멘돈카츠전문점 | 스트레스가 엄청난날! 매운라멘 한그릇하면 쏵 사라지는게 넘 좋아요 ㅎㅎ 그리고 맛있어요👍👍👍👍👍👍 가라아케도 처음에 먹었을때는 좀 그랬는데 요즘은 튀김옷도 바뀐것같고 부드럽고 바삭바삭한게 넘 맛있게 잘먹었습니다👍👍👍👍👍👍👍👍👍👍👍👍🍜🍗 | 5
모던니혼일본라멘돈카츠전문점 | 맛있게 잘먹었습니다 | 5
모던니혼일본라멘돈카츠전문점 | 리뷰 적으려고 30 분 기다렸습니다 배달이 훨씬 빨라서 먹다가 허겁지겁 ㅠ 라멘 양 생각보다 꽤 돼서 쉬엄쉬엄 먹는 중입니다 사진처럼 육수/면 따로라 하나도 안 불고 ㅠㅠ 차슈 너무 맛있어요 돈가스도 고기 두툼하고 좋아용 소스 배합 와사비 넣어서 느끼함 잡아 주는데 최고 ㅠㅠ 다음에도 또 시킬게용 개인적으로 차슈 추가 있었으면 하는 ㅠㅠ | 5
모던니혼일본라멘돈카츠전문점 |

In [29]:
f.close()

In [30]:
data = pd.read_csv('/Users/keonkookkim/Desktop/KKK/Selenium/valid_data.csv')
data.columns =['store','review','total_scope']
data

,store,review,total_scope
0,할매찜닭삼계탕,최악입니다 배달비 4000원 그렇다칩시다 이해합니다 죽하나 시킬라카면 추가금액 만원...,1
1,할매찜닭삼계탕,12000원짜리 죽 두개 시키는데 배달료 4000원추가 추가금액 1000원 총 29...,1
2,할매찜닭삼계탕,아니 이게 8천원 짜리밥이냐 양심이있어야지 김치고기덮밥이아니라 먹다말은거 볶아먹는맛...,1
3,할매찜닭삼계탕,1시간 반을 기다리다 안와서 전화드렸더니 그때서야 배달기사님이 안오신다고.. 못갈거...,1
4,할매찜닭삼계탕,배도 아프고 설사하고 운동하다가 대략난감 했네요. 배달 음식이지만 퀄리티좀 있게 포...,1
...,...,...,...
397,모던니혼일본라멘돈카츠전문점,맛있어요 배달은 보통정도?,4
398,모던니혼일본라멘돈카츠전문점,갠적으로 일본라멘 좋아하는데 배달하는곳은 첨이네요 불어서 올까봐 걱정했는데 국물따로...,5
399,모던니혼일본라멘돈카츠전문점,매운라멘이랑 돈까스 주문해서 먹었어요 돈까스 맛있네요 고기 두께는 두꺼운데 속은 촉...,5
400,모던니혼일본라멘돈카츠전문점,8.000원짜리 돈까스는 맛없어요. 고기 간이 안되서 싱겁고 쓰네요. 차슈 추가로 ...,1
